In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
df = pd.read_csv("churn_train.csv")
test = pd.read_csv("churn_test.csv")

In [3]:
df

,Unnamed: 0,age,marital_status,job_position,credit_sum,credit_length,scoring,education,tariff_id,region,average_region_wage,income,credit_count,overdue_count,churn
0,0,38.0,3.0,15.0,12340.91,12.0,0.365170,2.0,20.0,73.0,56000.0,43000.0,1.0,0.0,0
1,1,33.0,4.0,14.0,12118.00,10.0,0.507244,4.0,2.0,46.0,28160.0,20000.0,0.0,0.0,0
2,2,50.0,4.0,14.0,11691.00,6.0,0.382806,4.0,25.0,4.0,27390.0,12000.0,1.0,0.0,0
3,3,39.0,4.0,14.0,87467.00,12.0,0.508112,4.0,2.0,29.0,55880.0,80000.0,2.0,0.0,0
4,4,65.0,3.0,15.0,44879.00,10.0,0.491978,4.0,28.0,17.0,27060.0,45000.0,3.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128053,128053,45.0,3.0,15.0,45978.00,10.0,0.429848,4.0,2.0,22.0,22550.0,25000.0,3.0,0.0,0
128054,128054,59.0,4.0,10.0,27980.00,14.0,0.446021,2.0,1.0,62.0,27060.0,10500.0,0.0,0.0,0
128055,128055,27.0,3.0,14.0,22756.00,15.0,0.520948,4.0,2.0,46.0,28160.0,25000.0,0.0,0.0,0
128056,128056,36.0,3.0,14.0,55189.00,12.0,0.457322,2.0,2.0,63.0,45430.0,70000.0,2.0,0.0,0


In [4]:
df.describe()

,Unnamed: 0,age,marital_status,job_position,credit_sum,credit_length,scoring,education,tariff_id,region,average_region_wage,income,credit_count,overdue_count,churn
count,128058.000000,128058.000000,128053.000000,128058.000000,128057.000000,128058.000000,128055.000000,128055.000000,128058.000000,127910.00000,127910.000000,128057.000000,121183.000000,121183.000000,128058.000000
mean,64028.500000,36.502444,3.196872,13.153969,26108.496554,10.981469,0.469510,3.202077,14.136563,42.98653,32941.184661,40099.413503,2.104742,0.046178,0.174585
std,36967.304723,10.557304,0.739544,3.142273,16206.752729,3.533701,0.124283,1.064060,11.548063,22.24548,12584.991247,24737.953778,1.774634,0.211633,0.379613
min,0.000000,18.000000,1.000000,1.000000,2736.000000,3.000000,0.000000,1.000000,1.000000,1.00000,16830.000000,5000.000000,0.000000,0.000000,0.000000
25%,32014.250000,28.000000,3.000000,14.000000,14919.000000,10.000000,0.379602,2.000000,2.000000,25.00000,25520.000000,25000.000000,1.000000,0.000000,0.000000
50%,64028.500000,34.000000,3.000000,14.000000,21256.000000,10.000000,0.461519,4.000000,19.000000,40.00000,27390.000000,35000.000000,2.000000,0.000000,0.000000
75%,96042.750000,43.000000,4.000000,14.000000,32068.000000,12.000000,0.552381,4.000000,28.000000,63.00000,42460.000000,50000.000000,3.000000,0.000000,0.000000
max,128057.000000,71.000000,5.000000,18.000000,200000.000000,36.000000,1.128291,5.000000,32.000000,82.00000,66880.000000,950000.000000,21.000000,3.000000,1.000000


In [5]:
#missing data
df.isnull().mean()

Unnamed: 0             0.000000
age                    0.000000
marital_status         0.000039
job_position           0.000000
credit_sum             0.000008
credit_length          0.000000
scoring                0.000023
education              0.000023
tariff_id              0.000000
region                 0.001156
average_region_wage    0.001156
income                 0.000008
credit_count           0.053687
overdue_count          0.053687
churn                  0.000000
dtype: float64

In [6]:
X = df.drop(['churn', 'scoring','region','income'],axis=1).values
y = df['churn'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 42)

In [8]:
X_train= pd.DataFrame(X_train)
X_train.fillna(X_train.mean(), inplace=True)

In [9]:
randomforest = RandomForestRegressor(n_estimators=100)
randomforest.fit(X_train, y_train)

RandomForestRegressor()

In [12]:
X_test= pd.DataFrame(X_test)
X_test.fillna(X_test.mean(), inplace=True)

In [13]:
X_test

,0,1,2,3,4,5,6,7,8,9,10
0,33507.0,56.0,2.0,14.0,7979.00,6.0,4.0,2.0,33990.0,3.0,0.0
1,16990.0,58.0,3.0,14.0,5248.96,4.0,2.0,2.0,28160.0,2.0,0.0
2,91297.0,43.0,3.0,4.0,45369.00,12.0,2.0,4.0,45430.0,1.0,0.0
3,69279.0,40.0,3.0,14.0,16794.00,10.0,2.0,19.0,26620.0,2.0,0.0
4,2271.0,54.0,3.0,15.0,31465.00,10.0,4.0,19.0,26620.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
25607,24197.0,34.0,3.0,14.0,22489.00,12.0,5.0,2.0,27060.0,3.0,0.0
25608,14666.0,27.0,3.0,14.0,15990.00,8.0,5.0,28.0,56000.0,2.0,0.0
25609,30630.0,33.0,3.0,14.0,16990.00,3.0,2.0,20.0,26620.0,4.0,0.0
25610,62342.0,31.0,3.0,14.0,12990.00,10.0,4.0,28.0,27060.0,2.0,0.0


In [15]:
y_pred = randomforest.predict(X_test)
y_pred = np.around(y_pred, decimals=0)

In [16]:
f1_score(y_test, y_pred, average='weighted')

0.7846929578285947

# Валидация (Эрдэни)

In [17]:
df_true = pd.read_csv("churn_y_true.csv")
y_true = df_true['churn'].values
y_true

array([0, 1, 0, ..., 0, 0, 0])

In [18]:
df2 = pd.read_csv("churn_test.csv")

In [19]:
X2 = df2.drop(['scoring','region','income'],axis=1).values

In [20]:
X2= pd.DataFrame(X2)
X2.fillna(X2.mean(), inplace=True)

In [21]:
y_pred2 = randomforest.predict(X2)
y_pred2 = np.around(y_pred2, decimals=0)

In [22]:
f1_score(y_true, y_pred2, average='weighted')

0.7730635119571949